In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, IntegerType, BooleanType

In [2]:
# Define Kafka topic and broker
KAFKA_BROKER = "localhost:9092"
KAFKA_TOPIC = "wiki-changes"

In [3]:
# Initialize SparkSession with Kafka support
spark = SparkSession.builder \
    .appName("WikiChangeStream") \
    .master("spark://localhost:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3") \
    .config("spark.executor.memory", "1g") \
    .config("spark.executor.cores", "1") \
    .config("spark.cores.max", "2") \
    .getOrCreate()

:: loading settings :: url = jar:file:/Users/mpalamariuk/anaconda3/envs/mmds-project/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/mpalamariuk/.ivy2/cache
The jars for the packages stored in: /Users/mpalamariuk/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-af458f16-7dfd-44a6-9653-94d035bfec37;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.3 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 549ms :: artifacts

In [4]:
# Schema for Kafka JSON data
schema = StructType() \
    .add("id", StringType()) \
    .add("title", StringType()) \
    .add("timestamp", StringType()) \
    .add("user", StringType()) \
    .add("bot", BooleanType()) \
    .add("minor", BooleanType()) \
    .add("change", IntegerType()) \
    .add("comment", StringType())

24/11/18 01:17:08 ERROR TaskSchedulerImpl: Lost an executor 0 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:17:08 ERROR TaskSchedulerImpl: Lost an executor 1 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:17:12 ERROR TaskSchedulerImpl: Lost an executor 3 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:17:12 ERROR TaskSchedulerImpl: Lost an executor 2 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:17:16 ERROR TaskSchedulerImpl: Lost an executor 5 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:17:16 ERROR TaskSchedulerImpl: Lost an executor 4 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:17:20 ERROR TaskSchedulerImpl: Lost an executor 6 (already removed): Unable to cre

In [ ]:
# Read data from Kafka topic
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BROKER) \
    .option("subscribe", KAFKA_TOPIC) \
    .load()

# Parse JSON data from Kafka messages
parsed_df = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

# Output the data to the console
query = parsed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

In [ ]:
spark

In [ ]:
query.isActive

sh: spark-submit.sh: No such file or directory


24/11/18 01:19:53 ERROR TaskSchedulerImpl: Lost an executor 70 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:19:53 ERROR TaskSchedulerImpl: Lost an executor 71 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:19:57 ERROR TaskSchedulerImpl: Lost an executor 72 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:19:57 ERROR TaskSchedulerImpl: Lost an executor 73 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:20:01 ERROR TaskSchedulerImpl: Lost an executor 74 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:20:01 ERROR TaskSchedulerImpl: Lost an executor 75 (already removed): Unable to create executor due to Too large frame: 5785721462337832960
24/11/18 01:20:06 ERROR TaskSchedulerImpl: Lost an executor 76 (already removed): Unable